Reconstructing virtual markers
==============================

In this tutorial, we will reconstruct virtual markers for anatomic landmarks that were not physically instrumented during the movement acquisition. We usually do this kind of reconstruction when it is not practical or feasible to stick a marker on an anatomical landmark. Instead, we track a rigid body affixed to the whole segment, and express the position of this virtual marker relative to this rigid body.

This is a relatively complex process that requires these additional "calibration" acquisitions:

1. A static acquisition of a few seconds where we can see every marker.

2. Probing acquisitions, one for each virtual marker. In each of these acquisitions of a few seconds, we point the anatomical landmark using a calibrated probe. We need to see the probe's markers and the markers of the rigid body affixed to the landmark's segment.


This analysis is composed of the following steps:

**Calibration steps:**

**Step 1.** Define rigid body configurations (how are placed every markers one relative to the other on every rigid body);

**Step 2.** Define virtual marker configurations (where are the anatomical landmarks relative to these rigid bodies);

**Task analysis steps:**

**Step 3.** Reconstruct series of rigid body's reference frames during the analyzed task (where are every segment and how are they oriented in space);

**Step 4.** Reconstruct series of virtual markers during the analyzed tasks (where are the virtual markers relative to the segments' rigid bodies).

In [ ]:
import kineticstoolkit.lab as ktk
import numpy as np

Read and visualize marker trajectories
--------------------------------------

We proceed exactly as in the previous tutorials, but this time we will perform the analysis based on a minimal set of markers. For the right arm and forearm, all we have is one real marker on the lateral epicondyle, and three markers on two rigid bodies affixed to the arm and forearm segments (we will show every other in blue for easier visualization).

In [ ]:
# Read the markers
markers = ktk.kinematics.read_c3d_file(
    ktk.config.root_folder + '/data/kinematics/sample_propulsion.c3d')

# Set every unnecessary markers to blue
keep_white = ['LateralEpicondyleR', 'ArmR1', 'ArmR2', 'ArmR3',
        'ForearmR1', 'ForearmR2', 'ForearmR3']

for marker_name in markers.data:
    if marker_name not in keep_white:
        markers.add_data_info(marker_name, 'Color', 'b')

# Set the point of view for 3D visualization
viewing_options = {
    'zoom': 3.5,
    'azimuth': 0.8,
    'elevation': 0.16,
    'translation': (0.2, -0.7)
}

# Create the player
player = ktk.Player(markers, **viewing_options)
player.to_html5(start_time=0, stop_time=1)

Calibration: Defining rigid body configurations using a static acquisition
--------------------------------------------------------------------------

One of the aims of the static trial is to have a sample where every marker is visible. We use this trial to define the rigid body configuration. A rigid body configuration is a list of markers that form a rigid body, along with their local position in the rigid body's reference frame.

For this example, we will create reference frames for the rigid bodies 'ArmR' and 'ForearmR'.

In [ ]:
rigid_body_config = dict()  # Will contain the rigid body configurations.

# Read the static trial
static_markers = ktk.kinematics.read_c3d_file(
    ktk.config.root_folder + '/data/kinematics/sample_static.c3d')

# Show this trial, just to inspect it
player = ktk.Player(static_markers, **viewing_options)
player.to_html5(start_time=0, stop_time=0.5)

We can not create a rigid body configuration for the arm:

In [ ]:
rigid_body_config['ArmR'] = ktk.kinematics.create_rigid_body_config(
    static_markers, ['ArmR1', 'ArmR2', 'ArmR3'])

rigid_body_config['ArmR']

We proceed the same way for the forearm:

In [ ]:
rigid_body_config['ForearmR'] = ktk.kinematics.create_rigid_body_config(
    static_markers, ['ForearmR1', 'ForearmR2', 'ForearmR3'])

For the probe, we will create its rigid body configuration manually from its known specifications. Every 6 local point is expressed relative to a reference frame that is centered at the probe's tip:

In [ ]:
rigid_body_config['Probe'] = {
    'MarkerNames': ['Probe1', 'Probe2', 'Probe3',
                    'Probe4', 'Probe5', 'Probe6'],
    'LocalPoints': np.array([[
        [ 0.0021213, 0.0021213, 0.0020575, 0.0021213,  0.0017070,  0.0017762],
        [-0.0158328, 0.0158508, 0.0160096, 0.0161204, -0.0155780, -0.0156057],
        [ 0.0864285, 0.0864285, 0.1309445, 0.1754395,  0.1753805,  0.1308888],
        [ 1.,        1.,        1.,        1.,         1.,         1.       ]]]
    )
}

Now that we created these rigid body configurations, we will be able to track the position and orientation of these segments and probe in every other acquisition. This process can be done using the [kinematics.register_markers()](../api/kineticstoolkit.kinematics.rst#kineticstoolkit.kinematics.register_markers) function.

Although it is not necessary, let's see its action on the static acquisition we just processed.

In [ ]:
static_rigid_bodies = ktk.kinematics.register_markers(
    static_markers,
    rigid_body_config)

player = ktk.Player(static_markers, static_rigid_bodies, **viewing_options)
player.to_html5(start_time=0, stop_time=0.5)

Calibration: Defining the virtual marker configurations based on probing acquisitions
-------------------------------------------------------------------------------------

Now we will go though every probing acquisition and apply the same process on each acquisition:

1. Locate the probe and the segment's rigid body using [kinematics.register_markers()](../api/kineticstoolkit.kinematics.rst#kineticstoolkit.kinematics.register_markers) and the rigid body configurations we just created;

2. Express the tip of the probe in the segment's local coordinate system;

3. Create a virtual marker configuration using the probe tip's local position. For this step, we will be helped by the [kinematics.create_virtual_marker_config()](../api/kineticstoolkit.kinematics.rst#kineticstoolkit.kinematics.create_virtual_marker_config) function.

In [ ]:
virtual_marker_config = dict()  # Will contain all virtual marker configurations

Since this is a repetitive operation, we will create a new function that will be called for each probing acquisition:

In [ ]:
def process_probing_acquisition(file_name, rigid_body_name):

    # Load the markers
    probing_markers = ktk.kinematics.read_c3d_file(file_name)

    # Calculate rigid body trajectories using these markers
    probing_rigid_bodies = ktk.kinematics.register_markers(
        probing_markers, rigid_body_config)

    # Add the marker 'ProbeTip' in markers. This is the origin of the Probe
    # rigid body.
    probing_markers.data['ProbeTip'] = probing_rigid_bodies.data['Probe'][:, :, 3]

    # Create the marker configuration
    return ktk.kinematics.create_virtual_marker_config(
            probing_markers, probing_rigid_bodies, 'ProbeTip', rigid_body_name)

Now, we can process every probing acquisition.

In [ ]:
virtual_marker_config['AcromionR'] = process_probing_acquisition(
    ktk.config.root_folder +
    '/data/kinematics/sample_probing_acromion_R.c3d', 'ArmR')

virtual_marker_config['MedialEpicondyleR'] = process_probing_acquisition(
    ktk.config.root_folder +
    '/data/kinematics/sample_probing_medial_epicondyle_R.c3d', 'ArmR')

virtual_marker_config['RadialStyloidR'] = process_probing_acquisition(
    ktk.config.root_folder + 
    '/data/kinematics/sample_probing_radial_styloid_R.c3d', 'ForearmR')

virtual_marker_config['UlnarStyloidR'] = process_probing_acquisition(
    ktk.config.root_folder + 
    '/data/kinematics/sample_probing_ulnar_styloid_R.c3d', 'ForearmR')

For curiosity, let's see what is inside a virtual marker configuration:

In [ ]:
virtual_marker_config['AcromionR']

This is simply the position of the virtual marker in the `RigidBodyName` coordinate system.

Task analysis: Tracking the segments
------------------------------------

Now that we configured the rigid bodies and the virtual markers, we are ready to track the segments in the experimental trial we loaded at the beginning of this tutorial. To this effect, we will once again use the [kinematics.register_markers()](../api/kineticstoolkit.kinematics.rst#kineticstoolkit.kinematics.register_markers) function.

In [ ]:
# Compute the rigid body trajectories
rigid_bodies = ktk.kinematics.register_markers(markers, rigid_body_config)

# Show those rigid bodies and markers in a player
player = ktk.Player(markers, rigid_bodies, **viewing_options)

player.to_html5(start_time=0, stop_time=1)

Task analysis: Reconstructing the virtual markers
-------------------------------------------------

Now that we reconstructed the rigid bodies' coordinate systems during the whole acquisition, we will use these coordinate systems to reconstruct the virtual markers. We will show these virtual markers in cyan.

In [ ]:
for virtual_marker in virtual_marker_config:
    
    # Extract the local coordinates for this virtual marker
    local_coordinates = virtual_marker_config[virtual_marker]['LocalPoint']
    
    # Extract the coordinates system for its associate rigid body
    rigid_body_name = virtual_marker_config[virtual_marker]['RigidBodyName']
    reference_frame = rigid_bodies.data[rigid_body_name]
    
    # Get the global trajectory of this virtual marker
    global_coordinates = ktk.geometry.get_global_coordinates(
        local_coordinates, reference_frame)

    # Add this trajectory to the markers TimeSeries and color it cyan.
    markers.data[virtual_marker] = global_coordinates
    markers.add_data_info(virtual_marker, 'Color', 'c')


# Show the markers and rigid bodies in a player
player = ktk.Player(markers, rigid_bodies, **viewing_options)
player.to_html5(start_time=0, stop_time=1)

That is it, we reconstructed the acromion, medial epicondyle and both styloids from probing acquisitions,  without physical markers on these landmarks. We can conclude by adding the segments for clearer visualization. From now one, we could continue our analysis and calculate the elbow angles as in the previous tutorial.

In [ ]:
# Add the segments
segments = {
    'ArmR': {
        'Color': [1, 0.25, 0],
        'Links': [['AcromionR', 'MedialEpicondyleR'],
                  ['AcromionR', 'LateralEpicondyleR']]
    },
    'ForearmR': {
        'Color': [1, 0.5, 0],
        'Links': [['MedialEpicondyleR', 'RadialStyloidR'],
                  ['MedialEpicondyleR', 'UlnarStyloidR'],
                  ['LateralEpicondyleR', 'RadialStyloidR'],
                  ['LateralEpicondyleR', 'UlnarStyloidR'],
                  ['UlnarStyloidR', 'RadialStyloidR']]
    }
}

player = ktk.Player(markers, segments=segments, **viewing_options)
player.to_html5(start_time=0, stop_time=1)

For more information on kinematics, please check the [API Reference for the kinematics module](../api/kineticstoolkit.kinematics.rst).